In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

X, y = load_iris(return_X_y=True, as_frame=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X[:3]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2


In [2]:
y[:3]

0    0
1    0
2    0
Name: target, dtype: int64

In [3]:
from hypergbm.estimators import XGBoostEstimator
from hypernets.pipeline.base import Pipeline, DataFrameMapper
from hypernets.pipeline.transformers import MinMaxScaler, StandardScaler
from hypernets.core import OptimizeDirection
from hypernets.core.ops import ModuleChoice, HyperInput
from hypernets.core.search_space import HyperSpace
from hypernets.tabular.column_selector import column_number_exclude_timedelta


def search_space():
    space = HyperSpace()
    with space.as_default():
        input = HyperInput(name='input1')
        scaler_choice = ModuleChoice(
            [
                StandardScaler(name=f'numeric_standard_scaler'),
                MinMaxScaler(name=f'numeric_minmax_scaler')
            ], name=f'numeric_or_scaler'
        )
        num_pipeline = Pipeline([scaler_choice], name='numeric_pipeline', columns=column_number_exclude_timedelta)(input)
        union_pipeline = DataFrameMapper(default=None, input_df=True, df_out=True)([num_pipeline])
        xgb_est = XGBoostEstimator(fit_kwargs={})
        ModuleChoice([xgb_est], name='estimator_options')(union_pipeline)  # Make xgboost as a estimator choice
        space.set_inputs(input)
    return space

In [4]:
from hypergbm import HyperGBM
from hypernets.searchers import MCTSSearcher
rs = MCTSSearcher(search_space, max_node_space=10, optimize_direction=OptimizeDirection.Maximize)
hk = HyperGBM(rs, task='multiclass', reward_metric='accuracy', callbacks=[])
hk.search(X_train, y_train, X_eval=X_test, y_eval=y_test)